In [1]:
import os

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx',
 'crawler_10.xlsx',
 'crawler_100.xlsx',
 'crawler_101.xlsx',
 'crawler_102.xlsx',
 'crawler_103.xlsx',
 'crawler_104.xlsx',
 'crawler_105.xlsx',
 'crawler_106.xlsx',
 'crawler_107.xlsx',
 'crawler_11.xlsx',
 'crawler_12.xlsx',
 'crawler_13.xlsx',
 'crawler_14.xlsx',
 'crawler_15.xlsx',
 'crawler_16.xlsx',
 'crawler_17.xlsx',
 'crawler_18.xlsx',
 'crawler_19.xlsx',
 'crawler_2.xlsx',
 'crawler_20.xlsx',
 'crawler_21.xlsx',
 'crawler_22.xlsx',
 'crawler_23.xlsx',
 'crawler_24.xlsx',
 'crawler_25.xlsx',
 'crawler_26.xlsx',
 'crawler_27.xlsx',
 'crawler_28.xlsx',
 'crawler_29.xlsx',
 'crawler_3.xlsx',
 'crawler_30.xlsx',
 'crawler_31.xlsx',
 'crawler_32.xlsx',
 'crawler_33.xlsx',
 'crawler_34.xlsx',
 'crawler_35.xlsx',
 'crawler_36.xlsx',
 'crawler_37.xlsx',
 'crawler_38.xlsx',
 'crawler_39.xlsx',
 'crawler_4.xlsx',
 'crawler_40.xlsx',
 'crawler_41.xlsx',
 'crawler_42.xlsx',
 'crawler_43.xlsx',
 'crawler_44.xlsx',
 'crawler_45.xlsx',
 'crawler_46.xlsx',
 'crawler_47.xls

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/3.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：6


['1ATSH00013',
 '1ATSH00014',
 '1ATSH00012',
 '1ATSH00015',
 '1ATSH00018',
 '1ATSH00019']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(f'''https://www.1aauto.com/catalog/product/fit?skuId={input_.loc[a, 'Part Code']}&pathName={input_.loc[a, 'Url'].removeprefix('https://www.1aauto.com').replace('/', '%2F')}''',
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(resp.text, 'lxml')
html = etree.HTML(str(soup))

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div id="product-fit-tab">
   <hr class="hr"/>
   <div class="available-makes fitDropdown" id="available-makes">
    <h2>
     Fits these Makes
    </h2>
    <div class="available-makes-list expanderContainer fitContainer" id="available-makes-list">
     <div class="fit-makes-filter filter" data-filter-target=".fitTable" id="fit-makes-filter">
      <div class="floating-label-input-container">
       <div class="oneaDropdown" id="make-list">
        <span class="floating-label d-none">
         Select vehicle
        </span>
        <button class="dropdown-toggle">
        </button>
        <svg class="bi bi-triangle-fill svg-triangle" fill="none" height="5" viewbox="0 0 10 5" width="10" xmlns="http://www.w3.org/2000/svg">
         <path d="M5 4.5L0.5 0H9.5L5 4.5Z" fill="#333333">
         </path>
        </svg>
        <ul class="oneaDropdownMenu">
        </ul>
       </div>
      </div>
     </div>
     <div class="expander-container-button">
      <button class="ex

In [9]:
list_th = [text.strip() for text in html.xpath('//table[@class="fitTable sortableTable onea-table"]/thead/tr/th/text()')]

list_th

[]

In [10]:
list_td = [[text.strip() for text in tr.xpath('./td/text()')] for tr in html.xpath('//table[@class="fitTable sortableTable onea-table"]/tbody/tr')]

list_td

[]

In [11]:
list_oem = [text.strip() for text in html.xpath('//table[@id="interchange-codes-table"]/tbody/tr/td[2]/text()')]
oem = ';'.join(sorted(list(set(list_oem)), key=list_oem.index))

oem

''

In [12]:
df_temp = pd.DataFrame(list_td, columns=list_th)
df_temp['OEM'] = oem
df_temp['No'] = input_.loc[a, 'No']
df_temp['Url'] = input_.loc[a, 'Url']

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,OEM,No,Url


In [13]:
crawler_status = 'ok'

crawler_status

'ok'

In [14]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [15]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No', 'Make', 'Model', 'Year'],
                                                ascending=[True, True, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,OEM,No,Url


In [16]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
